In [1]:
'''
Method developed by Anoop Praturu: https://www.biorxiv.org/content/10.1101/2022.10.12.511940v1
Code from Anoop Praturu on Oct 2021
Move from pystan to cmdstanpy based on suggestions by Milo Julis
Edited by Mingchen Yao on May 26 2023
'''

import numpy as np
import diptest  
import matplotlib.pyplot as plt
import cmdstanpy as stan
import scipy.stats as stats
import pickle
import seaborn as sns
%matplotlib inline
import pandas as pd

import plotly.express as px
from matplotlib import rcParams
import plotly.express as px

In [ ]:
# # if there is anything wrong with comstan: re-install it. Otherwise don't run this cell
# from cmdstanpy import install_cmdstan
# install_cmdstan(overwrite = True)

In [ ]:
!python --version

# Functions for Generating Synthetic Data

In [2]:
#returns hyperbolic distance between vectors in poincare ball
def poincare_dist(v1, v2):
    sq = np.sum(np.square(v1-v2))
    r1 = np.sum(np.square(v1))
    r2 = np.sum(np.square(v2))
    inv = 2.0*sq/((1.0-r1)*(1.0-r2))
    return np.arccosh(1.0 + inv)

#return NxN symmetric distance matrix from poincare coordinates
def get_dmat(p_coords):
    N = p_coords.shape[0]
    dists = np.zeros((N, N))
    
    for i in np.arange(N):
        for j in np.arange(i+1, N):
            dists[i][j] = poincare_dist(p_coords[i], p_coords[j])
            dists[j][i] = dists[i][j]
    return dists

In [ ]:
#generate 100 data points uniformly distributed in 5 dimensional hyperbolic space of radius R=4
dim=5; R=4.0;
n=100
dirs = stats.norm.rvs(size=(n, dim))
dirs = (dirs.T/np.sqrt(np.sum(np.square(dirs), axis=1))).T
U = stats.uniform.rvs(size=n)
rs_p = np.tanh(np.log((1-np.exp(-R))/np.exp(-R)*(U) + 1.0)/2.0)
p_coords = rs_p.reshape(-1,1)*dirs

#add noise to the computed distance matrix to simulate a more realistic dataset
mat_dim = get_dmat(p_coords) + 0.05*R*stats.norm.rvs(size=(n,n))
for i in np.arange(n):
    for j in np.arange(i+1, n):
        mat_dim[j][i] = mat_dim[i][j]
mat_dim = 2.0*mat_dim/np.max(mat_dim)

# Code for fitting Bayesian Model

In [3]:
path = '/Users/iuliarusu/Documents/Sharpee/HMDS-example/model/'
ltz_m = stan.CmdStanModel(stan_file=path+'lorentz.stan')

## Run the optimizer given the synthetically generated Data set

In [4]:
#import AVA data from all cells
SMDD_0_df = pd.read_csv ('/Users/iuliarusu/Documents/Sharpee/Clustering/clustered_bacterial_stim/SMDD_0_df.csv')
SMDD_1_df = pd.read_csv('/Users/iuliarusu/Documents/Sharpee/Clustering/clustered_bacterial_stim/SMDD_1_df.csv') 
SMDD_2_df = pd.read_csv('/Users/iuliarusu/Documents/Sharpee/Clustering/clustered_bacterial_stim/SMDD_2_df.csv')
SMDD_3_df = pd.read_csv('/Users/iuliarusu/Documents/Sharpee/Clustering/clustered_bacterial_stim/SMDD_3_df.csv') 
SMDD_4_df = pd.read_csv('/Users/iuliarusu/Documents/Sharpee/Clustering/clustered_bacterial_stim/SMDD_4_df.csv') 
SMDD_5_df = pd.read_csv('/Users/iuliarusu/Documents/Sharpee/Clustering/clustered_bacterial_stim/SMDD_5_df.csv') 
SMDD_6_df = pd.read_csv('/Users/iuliarusu/Documents/Sharpee/Clustering/clustered_bacterial_stim/SMDD_6_df.csv')

In [5]:
SMDV_0_df = pd.read_csv ('/Users/iuliarusu/Documents/Sharpee/Clustering/clustered_bacterial_stim/SMDV_0_df.csv')
SMDV_1_df = pd.read_csv('/Users/iuliarusu/Documents/Sharpee/Clustering/clustered_bacterial_stim/SMDV_1_df.csv') 
SMDV_2_df = pd.read_csv('/Users/iuliarusu/Documents/Sharpee/Clustering/clustered_bacterial_stim/SMDV_2_df.csv')
SMDV_3_df = pd.read_csv('/Users/iuliarusu/Documents/Sharpee/Clustering/clustered_bacterial_stim/SMDV_3_df.csv') 
SMDV_4_df = pd.read_csv('/Users/iuliarusu/Documents/Sharpee/Clustering/clustered_bacterial_stim/SMDV_4_df.csv') 
SMDV_5_df = pd.read_csv('/Users/iuliarusu/Documents/Sharpee/Clustering/clustered_bacterial_stim/SMDV_5_df.csv') 
SMDV_6_df = pd.read_csv('/Users/iuliarusu/Documents/Sharpee/Clustering/clustered_bacterial_stim/SMDV_6_df.csv')

In [ ]:
#process all AVA cells with from the same stim, stim groups: 0, 1, 6 OR 2, 3, 4, 5
#stim 1
SMDD_0 = np.array(SMDD_0_df.iloc[:, 1: -2])
SMDD_1 = np.array(SMDD_1_df.iloc[:, 1: -3]) #this has an extra column for each cell's original id
SMDD_6 = np.array(SMDD_6_df.iloc[:, 1: -3]) #this has an extra column for each cell's original id

#concatenate by row
SMDD_stim_0 = np.concatenate((SMDD_0, SMDD_1, SMDD_6 ), axis=0)

In [ ]:
#same for SMDV
SMDV_0 = np.array(SMDV_0_df.iloc[:, 1: -2])
SMDV_1 = np.array(SMDV_1_df.iloc[:, 1: -3]) #this has an extra column for each cell's original id
SMDV_6 = np.array(SMDV_2_df.iloc[:, 1: -3]) #this has an extra column for each cell's original id

SMDV_stim_0 = np.concatenate((SMDV_0, SMDV_1, SMDV_6 ), axis=0)

In [ ]:
#concatenate AVA and SMDV
all_stim_0 = np.concatenate((SMDD_stim_0, SMDV_stim_0), axis=0)

In [ ]:
all_stim_0.shape

In [ ]:
all_stim_0_df = pd.DataFrame(all_stim_0)

# For processing both stimuli at once

In [6]:
SMDD_0 = np.array(SMDD_0_df.iloc[:, 1: -2])
SMDD_1 = np.array(SMDD_1_df.iloc[:, 1: -3]) 
SMDD_2 = np.array(SMDD_2_df.iloc[:, 1: -3])
SMDD_3 = np.array(SMDD_3_df.iloc[:, 1: -3])
SMDD_4 = np.array(SMDD_4_df.iloc[:, 1: -3])
SMDD_5 = np.array(SMDD_5_df.iloc[:, 1: -3])
SMDD_6 = np.array(SMDD_6_df.iloc[:, 1: -3]) 

#concatenate by row
SMDD_all_stim = np.concatenate((SMDD_0, SMDD_1, SMDD_2, SMDD_3, SMDD_4, SMDD_5, SMDD_6 ), axis=0)

In [7]:
#same for SMDV
SMDV_0 = np.array(SMDV_0_df.iloc[:, 1: -2])
SMDV_1 = np.array(SMDV_1_df.iloc[:, 1: -3])
SMDV_2 = np.array(SMDV_2_df.iloc[:, 1: -3])
SMDV_3 = np.array(SMDV_3_df.iloc[:, 1: -3])
SMDV_4 = np.array(SMDV_4_df.iloc[:, 1: -3])
SMDV_5 = np.array(SMDV_5_df.iloc[:, 1: -3]) 
SMDV_6 = np.array(SMDV_2_df.iloc[:, 1: -3]) 

SMDV_all_stim = np.concatenate((SMDV_0, SMDV_1, SMDV_2, SMDV_3, SMDV_4, SMDV_5, SMDV_6 ), axis=0)

In [8]:
#concatenate AVA and SMDV
all_stim = np.concatenate((SMDD_all_stim, SMDV_all_stim), axis=0)

In [9]:
all_stim_df = pd.DataFrame(all_stim)

In [10]:
corr_matrix = all_stim_df.T.corr()


In [11]:
distance_matrix = 1 - corr_matrix
distance_matrix_squaredp = (1 - corr_matrix**2) * 2

In [12]:
distance_matrix_squaredp.shape

(26, 26)

In [13]:
#dictionary to specify input to simulation

# dat={'N':100, 'D':5, 'deltaij':mat_dim}
dat = {'N': 26 , 'D': 3 , 'deltaij':distance_matrix_squaredp}
#run optimizer
model = ltz_m.optimize(data=dat, iter=250000, algorithm='LBFGS', tol_rel_grad=1e2)

09:48:18 - cmdstanpy - INFO - Chain [1] start processing
09:48:27 - cmdstanpy - INFO - Chain [1] done processing


In [14]:
# build result 
hyp_emb = {'euc':model.euc, 'sig':model.sig, 'lambda':model.stan_variable('lambda')}

# # and save
# fdname = './emb5d.pickle'
# with open(fdname,'wb') as file:
#     pickle.dump(hyp_emb, file,  protocol=pickle.HIGHEST_PROTOCOL)

09:49:34 - cmdstanpy - WARNING - The default behavior of CmdStanMLE.stan_variable() will change in a future release to always return a numpy.ndarray, even for scalar variables.


In [ ]:
# # load existing results
# fdname = './emb5d.pickle'
# with open(fdname, 'rb') as file:
#     hyp_emb = pickle.load(file)

## Some utility functions for post-processing the simulation output

In [15]:
def d_lor(t1, t2, E1, E2):
    return np.arccosh(t1*t2 - np.dot(E1, E2))

#returns embedding distance matrix from optimization fit
def get_embed_dmat(fit):
    N = fit['euc'].shape[0]
    fit_ts = np.sqrt(1.0 + np.sum(np.square(fit['euc']), axis=1))

    fit_mat = np.zeros((N, N))

    for i in np.arange(N):
        for j in np.arange(i+1,N):
            fit_mat[i][j] = d_lor(fit_ts[i], fit_ts[j], fit['euc'][i], fit['euc'][j])
            fit_mat[j][i] = fit_mat[i][j]
            
    return fit_mat

#return poincare coordinates
def get_poin(fit):
    ts = np.sqrt(1.0 + np.sum(np.square(fit['euc']), axis=1))
    return (fit['euc'].T / (ts + 1)).T

In [16]:
def process_sim(fit):
    fit['emb_mat'] = get_embed_dmat(fit)/fit['lambda']
    fit['pcoords'] = get_poin(fit)
    fit['radii'] = 2.0*np.arctanh(np.sqrt(np.sum(np.square(fit['pcoords']), axis=1)))

In [19]:
#run this function after running the optimizer to process the output data into more usable forms
process_sim(hyp_emb)

In [20]:
#curvature (or radius)
hyp_emb['lambda']

6.41017

In [21]:
#poincare embedding coordinates
hyp_emb['pcoords']

array([[ 0.51096545,  0.58419151, -0.62713135],
       [-0.69071447,  0.46324387,  0.55034174],
       [-0.67776413,  0.48921421,  0.54402729],
       [ 0.99018207, -0.05515661, -0.10810333],
       [ 0.98883649, -0.05561645, -0.11819745],
       [ 0.46298783, -0.87807918,  0.09669888],
       [ 0.45803423, -0.88030509,  0.09960221],
       [ 0.43358292,  0.07117845,  0.89572109],
       [-0.53635098,  0.65370638, -0.52935122],
       [-0.54174629,  0.65730059, -0.51939034],
       [ 0.29034882,  0.95317333,  0.05017934],
       [ 0.28748694,  0.95463114,  0.03560646],
       [-0.7465279 , -0.59032691, -0.29897306],
       [-0.76728328, -0.57949622, -0.26606726],
       [-0.80281736,  0.02150933,  0.59134841],
       [-0.80949993,  0.00660463,  0.58294756],
       [-0.82776005,  0.03847497,  0.55491822],
       [ 0.19390924, -0.35210191, -0.91266043],
       [ 0.19243721, -0.34288982, -0.91645853],
       [ 0.03848929, -0.91650764,  0.39193028],
       [ 0.01105581, -0.91160352,  0.405

In [22]:
SMDD_SMDV_pcoords_sphere = hyp_emb['pcoords']
%store SMDD_SMDV_pcoords_sphere

Stored 'SMDD_SMDV_pcoords_sphere' (ndarray)


In [ ]:
SMDD_SMDV_pcoords_15 = hyp_emb['pcoords']
%store SMDD_SMDV_pcoords_15

In [ ]:
#emebdding uncertainties
hyp_emb.keys()

# Shepard Diagram

In [ ]:
rcParams['font.family'] = 'sans-serif'
rcParams['font.sans-serif'] = 'Arial'

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(15,7.5))

ax[0].scatter(distance_matrix_squaredp.values[np.triu_indices(12, k=1)], hyp_emb['emb_mat'][np.triu_indices(12, k=1)], c='cornflowerblue') #cornflowerblue, lightcoral
ax[0].plot(np.arange(3), np.arange(3), c='black', linewidth=5, alpha=0.5)

ax[0].set_xlabel('Original Distances', fontsize=20)
ax[0].set_ylabel('Embedding Distances / $\lambda$', fontsize=20)

ax[1].hist(hyp_emb['sig'], color ='cornflowerblue') #cornflowerblue, lightcoral
ax[1].set_xlabel('Embedding Uncertainties', fontsize=20)

In [ ]:
hyperbolic_df= pd.DataFrame(hyp_emb['pcoords'], columns=['x', 'y', 'z'])



In [ ]:
column = ['SMDD'] * (SMDD_0.shape[0] + SMDD_1.shape[0] + SMDD_2.shape[0] + SMDD_3.shape[0] + SMDD_4.shape[0] + SMDD_5.shape[0] + SMDD_6.shape[0]) + ['SMDV'] * (SMDV_0.shape[0] + SMDV_1.shape[0] + SMDV_2.shape[0] + SMDV_3.shape[0] + SMDV_4.shape[0] + SMDV_5.shape[0] + SMDV_6.shape[0] )
SMDD_SMDV_labels = np.array(column)
hyperbolic_df['cluster labels'] = SMDD_SMDV_labels.T
hyperbolic_df

In [ ]:
%store SMDD_SMDV_labels 

In [ ]:


# Create a scatter plot
# fig = px.scatter_3d(data_frame = hyperbolic_df, x= 'x', y = 'y', z= 'z', title='SMDD and SMDV: Stimulus 0', color = 'cluster labels') #cornflowerblue, lightcoral

fig = px.scatter_3d(data_frame = hyperbolic_df, x= 'x', y = 'y', z= 'z', title='SMDD and SMDV Neurons: Bacterial Stimulus No. 0',  color ='cluster labels') #cornflowerblue, lightcoral
fig.update_layout(
    scene=dict(
        xaxis=dict(ticktext=[-1, -0.5, 0, 0.5, 1]),
        yaxis=dict(ticktext=[-1, -0.5, 0, 0.5, 1]),
        zaxis=dict(ticktext=[-1, -0.5, 0, 0.5, 1]),
    )
)

fig.show()

In [ ]:
# import plotly.express as px

# Example DataFrame, assuming hyperbolic_df is already defined with appropriate data
# Adding a column 'labels' to hyperbolic_df for the point labels if not already present
# hyperbolic_df['labels'] = ...

fig = px.scatter_3d(
    data_frame=hyperbolic_df, 
    x='x', 
    y='y', 
    z='z', 
    color='cluster labels', 
    # text='cluster labels',  # assuming 'labels' column contains the point labels
    # title='AVA and RME Neurons',
    labels={'x': 'X', 'y': 'Y', 'z': 'Z'},
    color_discrete_map={'SMDD': 'yellow', 'SMDV':'purple'}
)

fig.update_layout(
    scene=dict(
        xaxis=dict(
            tickvals=[-1, -0.5, 0, 0.5, 1],
            ticktext=['-1', '-0.5', '0', '0.5', '1'],
            title=dict(text='X', font=dict(size=20)),
            tickfont=dict(size=15),
            showgrid=False
        ),
        yaxis=dict(
            tickvals=[-1, -0.5, 0, 0.5, 1],
            ticktext=['-1', '-0.5', '0', '0.5', '1'],
            title=dict(text='Y', font=dict(size=20)),
            tickfont=dict(size=15),
            showgrid=False
        ),
        zaxis=dict(
            tickvals=[-1, -0.5, 0, 0.5, 1],
            ticktext=['-1', '-0.5', '0', '0.5', '1'],
            title=dict(text='Z', font=dict(size=20)),
            tickfont=dict(size=15),
            showgrid=False
        )
    ),
    legend=dict(
        title=dict(text='Cluster Labels', font=dict(size=12)),
        font=dict(size=10)
    ),
    # title=dict(
    #     text='SMDD and SMDV Neurons',
    #     font=dict(size=15)
    # ),
    margin=dict(l=20, r=20, b=90, t=40),
    autosize=True,
    width=700,
    height=500
)

# Customize marker size and opacity for better visualization
fig.update_traces(marker=dict(size=10, opacity=0.6))




fig.show()
fig.write_image("SMDD_SMDV.svg")


# Fitting For Dimension

Suppose we did not know a-priori that the data was 5D? This is usually the case

## Fit the model across a range of dimensions

In [ ]:
all_fits = []
for d in np.arange(2,15):
    dat={'N':26, 'D':d, 'deltaij':distance_matrix_squaredp}
    #run optimizer
    model = ltz_m.optimize(data=dat, iter=250000, algorithm='LBFGS', tol_rel_grad=1e2)
    all_fits.append({'euc':model.euc, 'sig':model.sig, 'lambda':model.stan_variable('lambda'), 't':model.time})




In [ ]:
model.stan_variable('lambda')

In [ ]:
#access the correct iteration 
iteration = all_fits[5]['t']


iteration

In [ ]:
model.time.shape

In [ ]:
#calculate radii based on time parameter
radii = np.arccosh(iteration)

In [ ]:
radii

In [ ]:
#Histogram of radii
fig, ax = plt.subplots(figsize=(15,7.5))

ax.hist(radii, color ='cornflowerblue') #cornflowerblue, lightcoral
ax.set_xlabel('Hyperbolic Embedding Radii', fontsize=20)

In [ ]:
#DIP analysis of radii
dip, pval = diptest.diptest(radii)

In [ ]:
dip, pval

In [ ]:
all_fits[0]['sig']

In [ ]:
#return negative log likelihood of fit
def MDS_lkl(fit, dmat):
    lkl = 0;
    N = fit['sig'].shape[0]
    
    sigs = fit['sig']
    lam = fit['lambda']
    emb_mat = get_embed_dmat(fit)
    
    for i in np.arange(N):
        for j in np.arange(i+1, N):
            seff = sigs[i]**2 + sigs[j]**2
            lkl += ((dmat[i][j] - emb_mat[i][j]/lam)**2 / (2.0*seff)) + 0.5*np.log(seff*2.0*np.pi)
    return lkl

#input: optimization fit and distance matrix
def BIC(fit, dmat):
    N,D = fit['euc'].shape
    n = 0.5*N*(N-1)
    k = N*D + N + 1.0 - 0.5*D*(D-1)
    
    return k*np.log(n) + 2.0*MDS_lkl(fit, dmat)

## Bayesian information criteria

There is a lot of formal Bayesian Theory behind this (see chapter 2 here https://urldefense.proofpoint.com/v2/url?u=https-3A__www.inference.org.uk_mackay_thesis.pdf&d=DwIGAg&c=-35OiAkTchMrZOngvJPOeA&r=B8GeUuyHfxQP8MseZuhipQ&m=KVww4gh9-XOtp1LqNUc0K-PGXOX3bm2QsokPFlBG9Vs&s=UBtNEdIXatq_zFpG53nmPCLbCnlgIgWigHYhEnBRYyo&e= ), but essentially we are trying to find the minimal number of parameters to describe a dataset. If the data is 5D, we don't want to use 7 parameters to describe it. The BIC is like a cost function that rewards a model that has a better fit to the likelihood function, but penalizes models that increase the number of their parameters. The model which minimizes this function will thus have the ideal trade-off of being able to model the data well without introducing too many parameters and overfitting the data.

In [ ]:
all_BIC = [BIC(fit, distance_matrix_squaredp) for fit in all_fits]

In [ ]:
all_BIC

As you can see the BIC is minimized at the true dimension of 5. Any more parameters would have been redundant, any less would not have properly fit the data

In [ ]:
fig, ax = plt.subplots(figsize=(7.5,7.5))

ax.scatter(np.arange(2,15), all_BIC, c = 'cornflowerblue') #cornflowerblue, lightcoral
ax.set_xlabel('Dimension', fontsize=20)
ax.set_ylabel('BIC', fontsize=20)

# Re-Running the optimizer
Lets start by generating some 2D data

In [ ]:
#generate 100 data points uniformly distributed in 5 dimensional hyperbolic space of radius R=4
dim=2; R=4.0;
n=100
dirs = stats.norm.rvs(size=(n, dim))
dirs = (dirs.T/np.sqrt(np.sum(np.square(dirs), axis=1))).T
U = stats.uniform.rvs(size=n)
rs_p = np.tanh(np.log((1-np.exp(-R))/np.exp(-R)*(U) + 1.0)/2.0)
p_coords = rs_p.reshape(-1,1)*dirs

#add noise to the computed distance matrix to simulate a more realistic dataset
mat_2D = get_dmat(p_coords) + 0.05*R*stats.norm.rvs(size=(n,n))
for i in np.arange(n):
    for j in np.arange(i+1, n):
        mat_2D[j][i] = mat_2D[i][j]
mat_2D = 2.0*mat_2D/np.max(mat_2D)

In [ ]:
#dictionary to specify input to simulation
dat={'N':100, 'D':2, 'deltaij':mat_2D}
#run optimizer
model2D = ltz_m.optimize(data=dat, iter=250000, algorithm='LBFGS', tol_rel_grad=1e2)
hyp_emb2D = {'euc':model2D.euc, 'sig':model2D.sig, 'lambda':model2D.stan_variable('lambda')}

In [ ]:
process_sim(hyp_emb2D)

#### Most pts are well fit, but a few are poorly fit (the ones with high sigma) which adds a lot of scatter to the shepard diagram
#### The optimizer got caught in a false minimum

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(15,7.5))

ax[0].scatter(mat_2D[np.triu_indices(100, k=1)], hyp_emb2D['emb_mat'][np.triu_indices(100, k=1)])
ax[0].plot(np.arange(3), np.arange(3), c='black', linewidth=5, alpha=0.5)

ax[0].set_xlabel('Original Distances', fontsize=20)
ax[0].set_ylabel('Embedding Distances / $\lambda$', fontsize=20)

ax[1].hist(hyp_emb2D['sig'])
ax[1].set_xlabel('Embedding Uncertainties', fontsize=20)

### Strategy: randomize the positions of the poorly fit points, and then return the coordinates as the initial conditions to continue optimizing. This bumps the simulation out of the false minimium without destroying all of the work its already done

In [ ]:
#figure out which pts have high uncertainty, and randomize their coordinates without touching the rest of the points
N_refit = np.where(hyp_emb2D['sig'] > 0.3)[0].shape[0]
hyp_emb2D['euc'][np.where(hyp_emb2D['sig'] > 0.3)] = stats.norm.rvs(size=(N_refit,2))

In [ ]:
#re-reun the optimizer, but this time sepcify the initial condition by passing the previous simulation dictionary
dat={'N':100, 'D':2, 'deltaij':mat_2D}
model2D = ltz_m.optimize(data=dat, iter=250000, algorithm='LBFGS', tol_rel_grad=1e2,inits = hyp_emb2D)
hyp_emb2D = {'euc':model2D.euc, 'sig':model2D.sig, 'lambda':model2D.stan_variable('lambda')}
# tst_2D = ltz_m.optimizing(data=dat, iter=250000, tol_rel_grad=1e2, init=tst_2D)

In [ ]:
process_sim(hyp_emb2D)

### We see that the fit is significantly better, although there are still a few poorly fit points. We could keep iterating the above process until all points are well fit.

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(15,7.5))

ax[0].scatter(mat_2D[np.triu_indices(100, k=1)], hyp_emb2D['emb_mat'][np.triu_indices(100, k=1)])
ax[0].plot(np.arange(3), np.arange(3), c='black', linewidth=5, alpha=0.5)

ax[0].set_xlabel('Original Distances', fontsize=20)
ax[0].set_ylabel('Embedding Distances / $\lambda$', fontsize=20)

ax[1].hist(hyp_emb2D['sig'])
ax[1].set_xlabel('Embedding Uncertainties', fontsize=20)